# Задача 12. Экспериментальное исследование алгоритмов решения задачи достижимости с КС ограничениями
# Автор: [Vladislav Miroshnikov](https://github.com/vladislav-miroshnikov)


## Введение
Пусть имеется размеченный ориентированный граф $G$. Будем интерпретировать его как конечный автомат. Таким образом, считаем, что граф $G$ (его представление в виде КА) задает регулярный язык $L_G$.

Пусть нам задана КС-грамматика $CFG$, задающая язык ограничений $L_{CFG}$.

Рассмотрим задачу $CFPQ = \{(v_i, v_j)|\exists\pi:w(v_i\pi v_j)\in L, v_i\in V_S, v_j\in V_F\}$, где $L = L_G\cap L_{CFG}$

Для языка $L_{CFG}$ строим рекурсивный автомат --- в нашем случае являющийся конечным автоматом над смешанным алфавитом грамматики.

## Постановка задачи
Задача посвящена анализу производительности различных алгоритмов решения задачи достижимости с контекстно-свободными ограничениями: алгоритма Хеллингса, матричного алгоритма, тензорного алгоритма. В ходе анализа необходимо
- Сравнить производительности реализаций различных алгоритмов для одной аппаратной платформы (CPU, GPU) и определить наиболее производительный алгоритм в рамках одной платформы.
- Сравнить производительности различных реализаций алгоритмов для разных платформ (CPU vs. GPU) и определить наиболее производительную реализацию.
- Сравнить производительности CFPQ алгоритмов со специализированными RPQ алгоритмами в задаче RPQ анализа и определить необходимость существования решения для частных случаев.

## Ход работы

- [x] Используя [pyCuBool](https://pypi.org/project/pycubool/) реализовать матричный алгоритм решения задачи достижимости с КС ограничениями. Аналогично тому, как это было сделано в домашней работе 10.
- [x] Используя [pyCuBool](https://pypi.org/project/pycubool/) реализовать тензорный алгоритм решения задачи достижимости с КС ограничениями. Аналогично тому, как это было сделано в домашней работе 11.
- [x] Подключить реализацию алгоритма Хеллингса из работы 9, матричного из 10 и тензорного из 11.
- [ ] Сформировать набор данных.
  - [x] Выбрать все графы из раздела [RDF](https://jetbrains-research.github.io/CFPQ_Data/dataset/RDF.html).
  - [ ] Запросы к RDF состоят из следующих блоков
    - Запросы из работы 5. Так как регулярные языки --- это подмножество контекстно-свободных, то решения, рассматриваемые в данной работе должны справляться с ними. Взять необходимо ровно те запросы, которые использовали именно вы. При этом, обратите внимание на то, что для разных алгоритмов нужен разный формат входа. Например, для тензорного грамматику из регулярного выражения можно получить просто сделав это выражение правой частью единственного правила.
    - Три классических запроса для анализа иерархии.
      - ```S -> subClassOf_r S subClassOf | type_r S type | subClassOf_r subClassOf | type_r type ```
      - ```S -> subClassOf_r S subClassOf | subClassOf ```
      - ```S -> broaderTransitive S broaderTransitive_r | broaderTransitive broaderTransitive_r ```
  - [ ] Выбрать графы bzip, gzip, ls, init, drivers, arch. Это графы, построенные по реальным программам и предназначенные для решения задач статического анализа.
  - [ ] Для статического анализа использовать единственный запрос. Обратите внимание, что он записан в максимально общем виде и непосредственно грамматика будет зависеть от алгоритма, который будет исследоваться.
    - ```
       M -> d_r V d
       V -> (M? a_r)∗ M? (a M?)∗
      ```
- [ ] С использованием сформированного набора данных провести сравнение производительности следующих решений
  - алгоритм Хеллингса
  - матричный алгоритм, реализованный с использованием sciPy
  - тензорный алгоритм, реализованный с использованием sciPy
  - матричный алгоритм, реализованный с использованием pyCuBool
  - тензорный алгоритм, реализованный с использованием pyCuBool
  - алгоритм выполнения регулярных запросов, реализованный с использованием pyCuBool (работа 5). Здесь только для RDF и регулярных запросов.
  - алгоритм выполнения регулярных запросов, реализованный с использованием sciPy. Здесь только для RDF и регулярных запросов.
- [ ] Оформить результаты экспериментов
- [ ] Провести анализ результатов
- [ ] Сделать notebook доступным по ссылке (обязательно выдать права на комментирование проверяющему)

## Исследуемые решения
Элементами матрицы смежности конечного автомата являются подмножества его меток. Чтобы не вводить операцию пересечения множеств в тензорном произведении, матрицы смежности двух конечных автоматов приводятся к виду булевых матриц.

Булевы матрицы на практике являются сильно разреженными. Сравниваемые алгоритмы используют различные библиотеки для работы с разреженными матрицами:
- [pyCuBool](https://pypi.org/project/pycubool/)
- [scipy.sparse](https://docs.scipy.org/doc/scipy/reference/sparse.html)

В scipy.sparce используются `dok` матрицы, в pyCuBool – встроенный класс матрицы, операции с объектами которого оптимизированы под GPU.

In [1]:
%%capture
# Установка библиотек в рабочее окружение
!pip install pycubool
# Установка проекта https://github.com/vladislav-miroshnikov/formal-lang-course
!git clone https://github.com/vladislav-miroshnikov/formal-lang-course

In [ ]:
%%capture
# Установка зависимостей проекта
!pip install -r formal-lang-course/requirements.txt

In [3]:
# Подключение директории проекта
import sys
sys.path.insert(1, 'formal-lang-course')

In [ ]:
RDF_GRAPH = (
    "skos",
    "generations",
    "travel",
    "univ_bench",
    "atom_primitive",
    "biomedical_mesure_primitive",
    "foaf",
    "people_pets",
    "funding",
    "wine",
    "pizza",
    "core",
    "pathways",
    "enzyme",
    "eclass_514en",
    "go_hierarchy",
    "go",
    "geospecies",
)
# Графы для статического анализа
STATIC_GRAPH = {
    "bzip", "gzip", "ls", "init", "drivers", "arch"
}
# Сокращения названий для удобного вывода
GRAPH_SHORT = {
    "skos": "skos",
    "generations": "generations",
    "travel": "travel",
    "univ_bench": "univ",
    "atom_primitive": "atom",
    "biomedical_mesure_primitive": "biomed",
    "foaf": "foaf",
    "people_pets": "p_pets",
    "funding": "funding",
    "wine": "wine",
    "pizza": "pizza",
    "core": "core",
    "pathways": "pathways",
    "enzyme": "enzyme",
    "eclass_514en": "eclass",
    "go_hierarchy": "go_hier",
    "go": "go",
    "geospecies": "geospec",
    "bzip": "bzip",
    "gzip": "gzip",
    "ls": "ls",
    "init": "init",
    "drivers": "drivers",
    "arch": "arch"
}

In [ ]:
# Информация о графах

from project.utils.graph_utils import get_graph_info
import pandas as pd 

edges = []
nodes = []

for graph_name in GRAPH_SHORT:
  graph_info = get_graph_info(graph_name)
  edges.append(graph_info.edges)
  nodes.append(graph_info.nodes)


df = pd.DataFrame(
    {
        "Edges": edges,
        "Nodes": nodes,
    },
    index=GRAPH_SHORT
)
df

file:///usr/local/lib/python3.7/dist-packages/cfpq_data/data/RDF/Graphs/<http:/sw.opencyc.org/concept/Mx4rvVi495wpEbGdrcN5Y29ycA> does not look like a valid URI, trying to serialize this will break.
file:///usr/local/lib/python3.7/dist-packages/cfpq_data/data/RDF/Graphs/<http:/sw.opencyc.org/concept/Mx4rvVi495wpEbGdrcN5Y29ycA> does not look like a valid URI, trying to serialize this will break.


GraphException: ignored

# Генерация данных

## Регулярные запросы

Так как регулярное множество лежит в классе КС, то алгоритмы $CFPQ$ должны работать и с регулярными ограничениями.

Все запросы к графам отвечают следующим шаблонам, где $L_i$ — метки графа:
- $(L_0 | L_1)^*\,L_2$
- $(L_0 | L_2)$+ ${L_1}^*$
- $L_0\,L_1\,L_2\,(L_3 | L_1)^*$
- $(L_0 | L_3)^*\,\,|\,\,(L_1 | L_2)^*$

### Крайний случай
Если в графе число меток меньше числа запросов, тогда метки дополняются уже имеющимися. (В графе должна быть по крайней мере 1 метка, иначе эксперимент не имеет смысла)

In [ ]:
# Генерация регулярных выражений

from pyformlang.regular_expression.regex_objects import Symbol
from pyformlang.regular_expression import Regex

NUMBER_OF_REGEX = 4

def regex_from_label(label):
  regex = Regex("")
  regex.head = Symbol(str(label))
  return regex

def expand(lst, expand_size):
    """
    If number of labels < NUMBER_OF_REGEX, then expand them with existing ones
    """
    expand_length = expand_size - len(lst)
    if expand_length <= 0:
        return lst
    expanded_lst = lst.copy()
    for i in range(expand_length):
        expanded_lst.append(expanded_lst[i])
    return expanded_lst

def regex_first(labels):
  """
  (l0 | l1)* l2
  """
  labels = expand(labels, NUMBER_OF_REGEX)
  regex_0 = regex_from_label(labels[0])
  regex_1 = regex_from_label(labels[1])
  regex_2 = regex_from_label(labels[2])
  return regex_0.union(regex_1).kleene_star().concatenate(regex_2)
  

def regex_second(labels):
  """
  (l0 | l2)+ l1*
  """
  labels = expand(labels, NUMBER_OF_REGEX)
  regex_0 = regex_from_label(labels[0])
  regex_1 = regex_from_label(labels[1])
  regex_2 = regex_from_label(labels[2])
  return regex_0.union(regex_2).union(regex_1.kleene_star())

def regex_third(labels):
  """
  l0 l1 l2 (l3 | l1)*
  """ 
  labels = expand(labels, NUMBER_OF_REGEX)
  regex_0 = regex_from_label(labels[0])
  regex_1 = regex_from_label(labels[1])
  regex_2 = regex_from_label(labels[2])
  regex_3 = regex_from_label(labels[3])
  return regex_0.concatenate(regex_1).concatenate(regex_2).concatenate((regex_3.union(regex_1)).kleene_star())

def regex_fourth(labels):
  """
  (l0 | l3)* | (l1 | l2)*
  """
  labels = expand(labels, NUMBER_OF_REGEX)
  regex_0 = regex_from_label(labels[0])
  regex_1 = regex_from_label(labels[1])
  regex_2 = regex_from_label(labels[2])
  regex_3 = regex_from_label(labels[3])
  left_regex = (regex_0.union(regex_3)).kleene_star()
  right_regex = (regex_1.union(regex_2)).kleene_star()
  return left_regex.union(right_regex)

REGEX_GENERATORS = (regex_first, regex_second, regex_third, regex_fourth)

## КС-запросы

### Три классических запроса для анализа иерархии

In [ ]:
def cfg_first():
  return CFG.from_text("S -> subClassOf_r S subClassOf | type_r S type | subClassOf_r subClassOf | type_r type")

def cfg_second():
  return CFG.from_text("S -> subClassOf_r S subClassOf | subClassOf")

def cfg_third():
  return CFG.from_text("S -> broaderTransitive S broaderTransitive_r | broaderTransitive broaderTransitive_r")

CFG_GENERATORS = (cfg_first, cfg_second, cfg_third)

### Запрос для статического анализа

In [ ]:
from project.grammar.ecfg import ECFG

def cfg_static():
  return ECFG.from_text(
      """
      M -> d_r V d
      V -> ((M | $) a_r)* (M | $) (a (M | $))*
      """
  )



# Алгоритмы $CFPQ$

## Реализация через *scipy.sparse*

In [ ]:
# Алгоритмы реализованы в предыдущих домашних заданиях
from project.cfpq.cfpq import cfpq_hellings as hellings_sparse 
from project.cfpq.cfpq import cfpq_matrix as matrix_sparse
from project.cfpq.cfpq import cfpq_tensor as tensor_sparse

## Реализация через *pycubool*

In [ ]:
from project.grammars.ecfg import ECFG
from project.utils.cfg_utils import transform_cfg_to_wcnf, transform_ecfg_to_rsm
from project.utils.automata_utils import transform_graph_to_nfa

import pycubool as cb

In [ ]:
def _matrix_based_cb(cfg, graph):
    wcnf = transform_cfg_to_wcnf(cfg)

    eps_prod_heads = [p.head.value for p in wcnf.productions if not p.body]
    term_productions = {p for p in wcnf.productions if len(p.body) == 1}
    var_productions = {p for p in wcnf.productions if len(p.body) == 2}
    nodes_num = graph.number_of_nodes()
    matrices = {
        v.value: cb.Matrix.empty(shape=(nodes_num, nodes_num)) for v in wcnf.variables
    }

    for i, j, data in graph.edges(data=True):
        l = data["label"]
        for v in {p.head.value for p in term_productions if p.body[0].value == l}:
            matrices[v][i, j] = True

    for i in range(nodes_num):
        for v in eps_prod_heads:
            matrices[v][i, i] = True

    any_changing = True
    while any_changing:
        any_changing = False
        for p in var_productions:
            old_nnz = set(matrices[p.head.value].to_list())
            matrices[p.head.value] = matrices[p.head.value].ewiseadd(
                matrices[p.body[0].value].mxm(matrices[p.body[1].value])
            )
            new_nnz = set(matrices[p.head.value].to_list())
            any_changing = any_changing or old_nnz != new_nnz

    return {
        (u, variable, v)
        for variable, matrix in matrices.items()
        for u, v in matrix.to_list()
    }

In [ ]:
from project.utils.rsm_pycubool import RSMMatrixCB

In [ ]:
def _tensor_based_cb(cfg, graph):
    graph_bm = RSMMatrixCB.from_nfa(transform_graph_to_nfa(graph))
    rsm = transform_ecfg_to_rsm((ECFG.from_pyformlang_cfg(cfg)))
    rsm_vars = {box.variable.value for box in rsm.boxes}
    rsm_bm = RSMMatrixCB.from_rsm(rsm)

    for eps_state in rsm_bm.start_states & rsm_bm.final_states:
        variable = rsm_bm.get_nonterminals(eps_state, eps_state)
        if variable not in graph_bm.bmatrix:
            graph_bm.bmatrix[variable] = cb.Matrix.empty(
                shape=(graph_bm.number_of_states, graph_bm.number_of_states)
            )
        for i in range(graph_bm.number_of_states):
            graph_bm.bmatrix[variable][i, i] = True

    tc = graph_bm.intersect(rsm_bm).get_transitive_closure()

    prev_nnz = tc.to_list()
    new_nnz = 0

    while prev_nnz != new_nnz:
        for i, j in tc.to_list():
            rsm_from = i % rsm_bm.number_of_states
            rsm_to = j % rsm_bm.number_of_states
            variable = rsm_bm.get_nonterminals(rsm_from, rsm_to)
            if not variable:
                continue
            graph_from = i // rsm_bm.number_of_states
            graph_to = j // rsm_bm.number_of_states
            if variable not in graph_bm.bmatrix:
                graph_bm.bmatrix[variable] = cb.Matrix.empty(
                    (graph_bm.number_of_states, graph_bm.number_of_states)
                )
            graph_bm.bmatrix[variable][graph_from, graph_to] = True

        tc = graph_bm.intersect(rsm_bm).get_transitive_closure()

        prev_nnz, new_nnz = new_nnz, tc.to_list()

    return {
        (u, label, v)
        for label, bm in graph_bm.bmatrix.items()
        if label in rsm_vars
        for u, v in bm.to_list()
    }

In [ ]:
from pyformlang.cfg import Variable

from project.cfpq.cfpq import _cfpq

In [ ]:
def matrix_cb(graph, cfg, start_nodes=None, final_nodes=None, start_var=Variable("S")):
  return _cfpq(graph, cfg, start_nodes, final_nodes, start_var, algorithm=_matrix_based_cb)

In [ ]:
def tensor_cb(graph, cfg, start_nodes=None, final_nodes=None, start_var=Variable("S")):
  return _cfpq(graph, cfg, start_nodes, final_nodes, start_var, algorithm=_tensor_based_cb)

### Небольшой тест, чтобы убедиться в корректности реализации алгоритмов на pycubool

In [ ]:
from project.utils.graph_utils import generate_two_cycles_graph
from pyformlang.cfg import CFG

graph = generate_two_cycles_graph("2", "1", "a", "b")
cfg = CFG.from_text(
"""
S -> A B
S -> A S1
S1 -> S B
A -> a
B -> b
""")

assert tensor_sparse(graph, cfg, None, None, "S") == tensor_cb(graph, cfg, None, None, "S")
assert matrix_sparse(graph, cfg, None, None, "S") == matrix_cb(graph, cfg, None, None, "S")